In [ ]:
import praw
import pandas as pd
import time

# Initialize Reddit instance
user_agent = "Scraper 1.0 by /u/python_engineer"
reddit = praw.Reddit(
    client_id="roVR4TckURjR22afdTBHeg",
    client_secret="EX48h9IBLKhO0U_7r7J12NbdIZXXHA",
    user_agent=user_agent
)

# Define team subreddits
team_subreddits = {
    "Arsenal": "Gunners",
    "Aston Villa": "AVFC",
    "Bournemouth": "AFCBournemouth",
    "Brentford": "Brentford",
    "Brighton": "BrightonHoveAlbion",
    "Chelsea": "ChelseaFC",
    "Crystal Palace": "CrystalPalace",
    "Everton": "Everton",
    "Fulham": "FulhamFC",
    "Liverpool": "LiverpoolFC",
    "Man City": "MCFC",
    "Man United": "RedDevils",
    "Newcastle": "NUFC",
    "Nottingham Forest": "NFFC",
    "Tottenham": "coys",
    "West Ham": "Hammers",
    "Wolves": "Wolves",
    "Leicester City": "lcfc",
    "Ipswich Town": "IpswichTownFC",
    "Southampton": "SaintsFC"
}

# Helper function to split dictionary into chunks
def chunk_dict(data, chunk_size):
    items = list(data.items())
    for i in range(0, len(items), chunk_size):
        yield dict(items[i:i + chunk_size])

# Process in batches of 2 teams
all_posts = []
chunk_size = 2
for batch_idx, batch in enumerate(chunk_dict(team_subreddits, chunk_size), 1):
    print(f"Processing batch {batch_idx}: {', '.join(batch.keys())}")
    for team, subreddit in batch.items():
        print(f"Processing {team}: {subreddit}")
        try:
            for submission in reddit.subreddit(subreddit).rising(limit=100):
                submission.comments.replace_more(limit=100)  # Partial comment expansion
                comments_with_replies = []

                for comment in submission.comments.list()[:100]:
                    replies = [reply.body for reply in comment.replies[:10]]  # Capture top replies
                    
                    comments_with_replies.append({
                        'comment': comment.body,
                        'replies': replies
                    })

                post = {
                    'team': team,
                    'title': submission.title,
                    'selftext': submission.selftext if submission.selftext else "",
                    'comments': comments_with_replies
                }
                all_posts.append(post)
                time.sleep(0.15)  # Prevent hitting rate limits
        except Exception as e:
            print(f"Error processing {team}: {e}")
            time.sleep(0.15)

# Flatten and save
flat_posts = []
for post in all_posts:
    for comment_data in post['comments']:
        flat_posts.append({
            'team': post['team'],
            'title': post['title'],
            'selftext': post['selftext'],
            'comment': comment_data['comment'],
            'replies': " ".join(comment_data['replies'])
        })

# Convert to DataFrame and save
df_all = pd.DataFrame(flat_posts)
df_all.to_csv("../data_raw/reddit_rising.csv", index=False)

print("All data saved to reddit_rising.csv")


Processing batch 1: Arsenal, Aston Villa
Processing Arsenal: Gunners
Processing Aston Villa: AVFC
Processing batch 2: Bournemouth, Brentford
Processing Bournemouth: AFCBournemouth
Processing Brentford: Brentford
Processing batch 3: Brighton, Chelsea
Processing Brighton: BrightonHoveAlbion
Processing Chelsea: ChelseaFC
Processing batch 4: Crystal Palace, Everton
Processing Crystal Palace: CrystalPalace
Processing Everton: Everton
Processing batch 5: Fulham, Liverpool
Processing Fulham: FulhamFC
Processing Liverpool: LiverpoolFC
Processing batch 6: Man City, Man United
Processing Man City: MCFC
Processing Man United: RedDevils
Processing batch 7: Newcastle, Nottingham Forest
Processing Newcastle: NUFC
Processing Nottingham Forest: NFFC
Processing batch 8: Tottenham, West Ham
Processing Tottenham: coys
Processing West Ham: Hammers
Processing batch 9: Wolves, Leicester City
Processing Wolves: Wolves
Processing Leicester City: lcfc
Processing batch 10: Ipswich Town, Southampton
Processing I